# Rutas

In [34]:
import os

# Carpeta base de trabajo
BASE_DIR = "/home/miguelo/molivares/PRUEBA 1"

# Mapping archivos de entrada → sufijo
archivos = {
    os.path.join(BASE_DIR, "tmy_Fuentes de Andalucia_España.csv"): "esp",
    os.path.join(BASE_DIR, "tmy_maria elena_Antofagasta.csv"): "chl",
    os.path.join(BASE_DIR, "tmy_Zhongwei_China.csv"): "chn"
}

# Carpeta de respaldo
carpeta_respaldo = os.path.join(BASE_DIR, "archivos_TMY")
os.makedirs(carpeta_respaldo, exist_ok=True)

# Carpeta base de trabajo
BASE_DIR = "/home/miguelo/molivares/PRUEBA 1"

# Archivos meteorológicos (rutas nuevas)
archivos = {
    "ESP": os.path.join(BASE_DIR, "esp_sam.csv"),
    "CHL": os.path.join(BASE_DIR, "chl_sam.csv"),
    "CHN": os.path.join(BASE_DIR, "chn_sam.csv")
}

# Ruta de guardado de resultados
archivo_resultados = os.path.join(BASE_DIR, "resultados_configuracion.csv")


# Parámetros usados en la simulación
capacidades = [100, 500, 1000, 5000, 10000]
dc_ratios   = [1.0, 1.1, 1.2, 1.3, 1.4]

# Archivo de resumen por tilt
archivo_por_tilt = os.path.join(BASE_DIR, "resultados_por_tilt.csv")

# Latitudes por país (valores fijos para la clasificación)
latitudes = {
    "ESP": 37.5,
    "CHL": 22.3,
    "CHN": 37.0
}

# Archivo de resultados
archivo_resultados = os.path.join(BASE_DIR, "resultados_configuracion.csv")

# Carpeta donde se guardarán los gráficos (ahora "graficos PV")
base_path    = os.path.join(BASE_DIR, "graficos PV")
lineas_path  = os.path.join(base_path, "lineas")
barras_path  = os.path.join(base_path, "barras")

# Crear carpetas si no existen
os.makedirs(lineas_path, exist_ok=True)
os.makedirs(barras_path, exist_ok=True)

# Latitudes oficiales por país (para clasificar Tilt)
latitudes = {
    "ESP": 37.5,
    "CHL": 22.3,
    "CHN": 37.0
}

# Orden de categorías de inclinación
orden_tilts = ["lat -10", "lat -5", "lat", "lat +5", "lat +10"]

# Función para clasificar cada Tilt según su latitud
def clasificar_tilt(row):
    lat = latitudes[row["País"]]
    if abs(row["Tilt"] - (lat - 10)) < 0.2:
        return "lat -10"
    elif abs(row["Tilt"] - (lat - 5)) < 0.2:
        return "lat -5"
    elif abs(row["Tilt"] - lat) < 0.2:
        return "lat"
    elif abs(row["Tilt"] - (lat + 5)) < 0.2:
        return "lat +5"
    elif abs(row["Tilt"] - (lat + 10)) < 0.2:
        return "lat +10"
    else:
        return "otro"

# Cargar datos y aplicar clasificación
df = pd.read_csv(archivo_resultados)
df["Tilt_type"] = df.apply(clasificar_tilt, axis=1)
df["Tilt_type"] = pd.Categorical(df["Tilt_type"], categories=orden_tilts, ordered=True)

# Archivos de recurso solar (SAM) para CSP
archivos_recurso = {
    "ESP": os.path.join(BASE_DIR, "esp_sam.csv"),
    "CHL": os.path.join(BASE_DIR, "chl_sam.csv"),
    "CHN": os.path.join(BASE_DIR, "chn_sam.csv")
}

# Potencia fija en kW para la planta CSP (100 MW)
potencia_kw = 100_000

# Horas de almacenamiento a simular (4 h a 18 h)
horas_almacenamiento = list(range(4, 19))

# Carpeta donde guardar los resultados
carpeta_resultados_csp = BASE_DIR
os.makedirs(carpeta_resultados_csp, exist_ok=True)

# Archivo de resultados CSP
archivo_resultados_csp = os.path.join(BASE_DIR, "resultados_csp.csv")

# Carpeta donde se guardarán los gráficos CSP
carpeta_graficos_csp = os.path.join(BASE_DIR, "graficos_csp_lcoe")
os.makedirs(carpeta_graficos_csp, exist_ok=True)

# En lugar de resultados_pv_lcoe.csv usamos el CSV de configuraciones PV
ruta_pv      = os.path.join(BASE_DIR, "resultados_configuracion.csv")
# Y tu CSP ya renombrado a resultados_csp.csv
ruta_csp     = os.path.join(BASE_DIR, "resultados_csp.csv")

# Carpeta donde pondremos los gráficos de sensibilidad
ruta_graficos = os.path.join(BASE_DIR, "graficos_sensibilidad")
os.makedirs(ruta_graficos, exist_ok=True)

# 1.Selección de Países y Recolección de Datos:

In [3]:
import pandas as pd
import shutil

for archivo, sufijo in archivos.items():
    # Leer encabezado de ubicación
    with open(archivo, 'r', encoding='utf-8') as f:
        lat = float(next(f).split(":")[1].strip())
        lon = float(next(f).split(":")[1].strip())
        elev = float(next(f).split(":")[1].strip())
        tz  = float(next(f).split(":")[1].strip())

    # Leer datos horarios desde fila 18
    df = pd.read_csv(archivo, skiprows=17, encoding='utf-8')

    # Filtrar fechas válidas con formato 'YYYYMMDD:HHMM'
    df = df[df['time(UTC)'].str.match(r'^\d{8}:\d{4}$', na=False)]

    # Convertir columna de tiempo
    df['FechaHora'] = pd.to_datetime(df['time(UTC)'], format='%Y%m%d:%H%M')
    df['Year']   = df['FechaHora'].dt.year
    df['Month']  = df['FechaHora'].dt.month
    df['Day']    = df['FechaHora'].dt.day
    df['Hour']   = df['FechaHora'].dt.hour
    df['Minute'] = df['FechaHora'].dt.minute

    # Renombrar columnas
    df.rename(columns={
        'G(h)':  'GHI',
        'Gb(n)': 'DNI',
        'Gd(h)': 'DHI',
        'T2m':   'Tdry',
        'RH':    'RH',
        'SP':    'Pres',
        'WS10m': 'Wspd',
        'WD10m': 'Wdir'
    }, inplace=True)

    # DataFrame final
    df_sam = df[['Year','Month','Day','Hour','Minute',
                 'GHI','DNI','DHI','Tdry','RH','Pres','Wspd','Wdir']].copy()

    # Encabezado NSRDB/SAM
    header1 = "Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation\n"
    header2 = f"PVGIS,00000,{sufijo.upper()}_Site,NA,{sufijo.upper()},{lat},{lon},{int(tz)},{int(elev)}\n"
    header3 = "Year,Month,Day,Hour,Minute,GHI,DNI,DHI,Tdry,RH,Pres,Wspd,Wdir\n"

    # Archivo de salida
    archivo_salida = os.path.join(BASE_DIR, f"{sufijo}_sam.csv")

    with open(archivo_salida, 'w', encoding='utf-8') as f_out:
        f_out.write(header1)
        f_out.write(header2)
        f_out.write(header3)
        df_sam.to_csv(f_out, index=False, header=False)

    # Copiar al respaldo
    archivo_respaldo = os.path.join(carpeta_respaldo, f"{sufijo}_sam.csv")
    shutil.copy2(archivo_salida, archivo_respaldo)

    print(f"✅ Archivo generado: {archivo_salida}")
    print(f"📁 Copia respaldada en: {archivo_respaldo}")


✅ Archivo generado: /home/miguelo/molivares/PRUEBA 1/esp_sam.csv
📁 Copia respaldada en: /home/miguelo/molivares/PRUEBA 1/archivos_TMY/esp_sam.csv
✅ Archivo generado: /home/miguelo/molivares/PRUEBA 1/chl_sam.csv
📁 Copia respaldada en: /home/miguelo/molivares/PRUEBA 1/archivos_TMY/chl_sam.csv
✅ Archivo generado: /home/miguelo/molivares/PRUEBA 1/chn_sam.csv
📁 Copia respaldada en: /home/miguelo/molivares/PRUEBA 1/archivos_TMY/chn_sam.csv


# 2.	Simulación de Plantas PV y CSP:

In [6]:
import PySAM.Pvwattsv7 as pv
import PySAM.Lcoefcr as Lcoefcr
import pandas as pd

# Parámetros generales
capacidades = [100, 500, 1000, 5000, 10000]
dc_ratios = [1.0, 1.1, 1.2, 1.3, 1.4]
capital_cost = 1_000_000
fixed_charge_rate = 0.07
fixed_operating_cost = 50000
variable_operating_cost = 0.01

resultados = []

def extraer_latitud(archivo):
    with open(archivo, 'r', encoding='utf-8') as f:
        # Saltamos la primera línea y leemos la segunda
        next(f)
        lat_line = next(f)
    return float(lat_line.split(",")[5])

# Loop principal
for pais, ruta_sam in archivos.items():
    lat = abs(extraer_latitud(ruta_sam))
    lat_round = round(lat, 1)

    inclinaciones = [
        round(max(0, min(lat_round - 10, 90)), 1),
        round(max(0, min(lat_round - 5, 90)), 1),
        lat_round,
        round(max(0, min(lat_round + 5, 90)), 1),
        round(max(0, min(lat_round + 10, 90)), 1)
    ]

    azimuts = [90, 135, 180, 225, 270] if extraer_latitud(ruta_sam) > 0 else [90, 45, 0, 315, 270]

    print(f"📌 {pais} - Latitud: {lat_round} → Tilts: {inclinaciones}")
    print(f"   Azimuts: {azimuts}")

    for capacidad in capacidades:
        for dcac in dc_ratios:
            for tilt in inclinaciones:
                for azimuth in azimuts:
                    try:
                        modelo = pv.new()
                        modelo.SolarResource.solar_resource_file = ruta_sam

                        # Diseño del sistema
                        modelo.SystemDesign.system_capacity = capacidad
                        modelo.SystemDesign.dc_ac_ratio   = dcac
                        modelo.SystemDesign.module_type   = 0
                        modelo.SystemDesign.array_type    = 1
                        modelo.SystemDesign.tilt          = tilt
                        modelo.SystemDesign.azimuth       = azimuth
                        modelo.SystemDesign.gcr           = 0.4
                        modelo.SystemDesign.inv_eff       = 96
                        modelo.SystemDesign.losses        = 14.0

                        modelo.execute()
                        energia_kwh = modelo.Outputs.annual_energy
                        energia_mwh = round(energia_kwh / 1000, 2)

                        modelo_lcoe = Lcoefcr.new()
                        modelo_lcoe.SimpleLCOE.annual_energy        = energia_kwh
                        modelo_lcoe.SimpleLCOE.capital_cost         = capital_cost
                        modelo_lcoe.SimpleLCOE.fixed_charge_rate    = fixed_charge_rate
                        modelo_lcoe.SimpleLCOE.fixed_operating_cost = fixed_operating_cost
                        modelo_lcoe.SimpleLCOE.variable_operating_cost = variable_operating_cost
                        modelo_lcoe.execute()
                        lcoe = round(modelo_lcoe.Outputs.lcoe_fcr, 4)

                        resultados.append({
                            "País": pais,
                            "Capacidad_kW": capacidad,
                            "DC_AC_Ratio": dcac,
                            "Tilt": tilt,
                            "Azimuth": azimuth,
                            "Energía_MWh": energia_mwh,
                            "LCOE_$_kWh": lcoe
                        })
                    except Exception as e:
                        print(f"⚠️ Error en {pais} | {capacidad}kW | Tilt {tilt}° | Azimuth {azimuth}° → {e}")

    print(f"✅ Simulaciones completadas para {pais}")

# Guardar resultados
df_resultados = pd.DataFrame(resultados)
df_resultados.sort_values(by=["País", "LCOE_$_kWh"], inplace=True)
df_resultados.to_csv(archivo_resultados, index=False)
print(f"\n📁 Resultados guardados en: {archivo_resultados}")

# Filtro: inclinación = latitud
latitudes = {pais: round(abs(extraer_latitud(ruta)), 1) for pais, ruta in archivos.items()}
df_filtrado = df_resultados[df_resultados.apply(lambda row: row["Tilt"] == latitudes[row["País"]], axis=1)]

print("\n📈 Resumen para Tilt = Latitud:\n")
resumen = df_filtrado.groupby(["País", "Capacidad_kW"]).agg({
    "Energía_MWh": "mean",
    "LCOE_$_kWh":  "mean"
}).round(2)
print(resumen.reset_index().to_string(index=False))


📌 ESP - Latitud: 37.5 → Tilts: [27.5, 32.5, 37.5, 42.5, 47.5]
   Azimuts: [90, 135, 180, 225, 270]
✅ Simulaciones completadas para ESP
📌 CHL - Latitud: 22.3 → Tilts: [12.3, 17.3, 22.3, 27.3, 32.3]
   Azimuts: [90, 45, 0, 315, 270]
✅ Simulaciones completadas para CHL
📌 CHN - Latitud: 37.0 → Tilts: [27.0, 32.0, 37.0, 42.0, 47.0]
   Azimuts: [90, 135, 180, 225, 270]
✅ Simulaciones completadas para CHN

📁 Resultados guardados en: /home/miguelo/molivares/PRUEBA 1/resultados_configuracion.csv

📈 Resumen para Tilt = Latitud:

País  Capacidad_kW  Energía_MWh  LCOE_$_kWh
 CHL           100       200.73        0.61
 CHL           500      1003.67        0.13
 CHL          1000      2007.35        0.07
 CHL          5000     10036.75        0.02
 CHL         10000     20073.50        0.02
 CHN           100       147.65        0.84
 CHN           500       738.24        0.18
 CHN          1000      1476.49        0.09
 CHN          5000      7382.44        0.03
 CHN         10000     14764.89    

In [8]:
import pandas as pd
import itertools

# Cargar el DataFrame de resultados
df = pd.read_csv(archivo_resultados)

# Función para extraer latitud del archivo SAM
def extraer_latitud(path):
    with open(path, 'r', encoding='utf-8') as f:
        next(f)                  # saltar línea 1
        lat_line = next(f)       # línea 2 con latitud
    return abs(float(lat_line.split(",")[5]))

# Tilts por país (latitud -10, -5, lat, +5, +10)
tilts_por_pais = {
    pais: [
        round(max(0, min(lat - 10, 90)), 1),
        round(max(0, min(lat - 5, 90)), 1),
        round(lat, 1),
        round(max(0, min(lat + 5, 90)), 1),
        round(max(0, min(lat + 10, 90)), 1)
    ]
    for pais, lat in {p: extraer_latitud(r) for p, r in archivos.items()}.items()
}

# Azimuts según hemisferio
azimuts_hemisferio = {
    "ESP": [90, 135, 180, 225, 270],   # Norte
    "CHN": [90, 135, 180, 225, 270],   # Norte
    "CHL": [90, 45, 0, 315, 270]       # Sur
}

# Verificar combinaciones faltantes
faltantes = []
for pais in archivos:
    for (cap, dcac, tilt, az) in itertools.product(
        capacidades,
        dc_ratios,
        tilts_por_pais[pais],
        azimuts_hemisferio[pais]
    ):
        existe = (
            (df["País"]        == pais) &
            (df["Capacidad_kW"]== cap)  &
            (df["DC_AC_Ratio"] == dcac)&
            (df["Tilt"].round(1)== round(tilt,1)) &
            (df["Azimuth"]     == az)
        ).any()
        if not existe:
            faltantes.append((pais, cap, dcac, tilt, az))

# Mostrar resumen
if faltantes:
    print(f"❌ FALTAN {len(faltantes)} combinaciones:")
    for p, c, d, t, a in faltantes:
        print(f" - País: {p}, Capacidad: {c} kW, DC/AC: {d}, Tilt: {t}°, Azimuth: {a}°")
else:
    print("✅ Todas las combinaciones están presentes.")


✅ Todas las combinaciones están presentes.


In [13]:
import pandas as pd

# Cargar el DataFrame de resultados
df = pd.read_csv(archivo_resultados)

# Función para clasificar cada tilt según la latitud de su país
def clasificar_tilt(row):
    lat = latitudes[row["País"]]
    if abs(row["Tilt"] - (lat - 10)) < 0.2:
        return "lat -10"
    elif abs(row["Tilt"] - (lat - 5)) < 0.2:
        return "lat -5"
    elif abs(row["Tilt"] - lat) < 0.2:
        return "lat"
    elif abs(row["Tilt"] - (lat + 5)) < 0.2:
        return "lat +5"
    elif abs(row["Tilt"] - (lat + 10)) < 0.2:
        return "lat +10"
    else:
        return "otro"

# Aplicar la clasificación
df["Tilt_type"] = df.apply(clasificar_tilt, axis=1)

# Agrupar solo los categorias válidas y calcular promedios
resumen_tilt = (
    df[df["Tilt_type"] != "otro"]
    .groupby(["País", "Capacidad_kW", "Tilt_type"])
    [["Energía_MWh", "LCOE_$_kWh"]]
    .mean()
    .round(2)
    .reset_index()
)

# Mostrar en pantalla
print("📊 Comparación de inclinación por país y capacidad:\n")
print(resumen_tilt.to_string(index=False))

# Guardar el CSV de salida
resumen_tilt.to_csv(archivo_por_tilt, index=False)
print(f"\n📁 Resultados exportados a: {archivo_por_tilt}")



📊 Comparación de inclinación por país y capacidad:

País  Capacidad_kW Tilt_type  Energía_MWh  LCOE_$_kWh
 CHL           100       lat       200.73        0.61
 CHL           100   lat +10       194.08        0.64
 CHL           100    lat +5       197.82        0.62
 CHL           100   lat -10       204.18        0.60
 CHL           100    lat -5       202.83        0.60
 CHL           500       lat      1003.67        0.13
 CHL           500   lat +10       970.38        0.14
 CHL           500    lat +5       989.12        0.13
 CHL           500   lat -10      1020.91        0.13
 CHL           500    lat -5      1014.15        0.13
 CHL          1000       lat      2007.35        0.07
 CHL          1000   lat +10      1940.76        0.07
 CHL          1000    lat +5      1978.23        0.07
 CHL          1000   lat -10      2041.82        0.07
 CHL          1000    lat -5      2028.30        0.07
 CHL          5000       lat     10036.75        0.02
 CHL          5000   lat +10  

In [17]:
import seaborn as sns
import matplotlib.pyplot as plt

# Para que seaborn use tu estilo
sns.set(style="whitegrid")

# 1) Gráficos de líneas: LCOE promedio vs Capacidad, una línea por país, uno por Tilt_type
for tilt in orden_tilts:
    df_tilt = df[df["Tilt_type"] == tilt]
    # Agrupar para obtener LCOE promedio por (Capacidad, País)
    df_line = (
        df_tilt
        .groupby(["Capacidad_kW", "País"])["LCOE_$_kWh"]
        .mean()
        .reset_index()
    )

    plt.figure(figsize=(10,6))
    for pais, grupo in df_line.groupby("País"):
        plt.plot(
            grupo["Capacidad_kW"],
            grupo["LCOE_$_kWh"],
            label=pais,
            linestyle=estilos_paises[pais]["linestyle"],
            marker=estilos_paises[pais]["marker"],
            linewidth=2
        )
    plt.title(f"LCOE Promedio vs Capacidad de Planta — Tilt: {tilt}")
    plt.xlabel("Capacidad (kW)")
    plt.ylabel("LCOE [$ / kWh]")
    plt.xscale("log")  # opcional: escala log para mejor visualización
    plt.legend(title="País")
    plt.grid(True, which="both", ls="--", lw=0.5)
    plt.tight_layout()
    archivo_line = os.path.join(lineas_path, f"lcoe_vs_cap_{tilt.replace(' ', '_')}.png")
    plt.savefig(archivo_line, dpi=300)
    print(f"✅ Guardado gráfico de línea (promedio): {archivo_line}")
    plt.close()

# 2) Gráficos de barras: Energía y LCOE promedio vs Tilt_type para cada Capacidad_kW
capacidades = sorted(df["Capacidad_kW"].unique())
for capacidad in capacidades:
    df_cap = df[df["Capacidad_kW"] == capacidad]
    resumen = (
        df_cap
        .groupby(["Tilt_type", "País"])[["Energía_MWh", "LCOE_$_kWh"]]
        .mean()
        .reset_index()
    )

    # –– Barras de Energía Promedio
    plt.figure(figsize=(10,6))
    sns.barplot(
        data=resumen,
        x="Tilt_type",
        y="Energía_MWh",
        hue="País",
        order=orden_tilts
    )
    plt.title(f"Energía Promedio vs Inclinación — {capacidad} kW")
    plt.xlabel("Inclinación")
    plt.ylabel("Energía Promedio [MWh]")
    plt.legend(title="País")
    plt.tight_layout()
    archivo_bar_e = os.path.join(barras_path, f"energia_{capacidad}kW_bar.png")
    plt.savefig(archivo_bar_e, dpi=300)
    print(f"✅ Guardado barras de Energía: {archivo_bar_e}")
    plt.close()

    # –– Barras de LCOE Promedio
    plt.figure(figsize=(10,6))
    sns.barplot(
        data=resumen,
        x="Tilt_type",
        y="LCOE_$_kWh",
        hue="País",
        order=orden_tilts
    )
    plt.title(f"LCOE Promedio vs Inclinación — {capacidad} kW")
    plt.xlabel("Inclinación")
    plt.ylabel("LCOE Promedio [$ / kWh]")
    plt.legend(title="País")
    plt.tight_layout()
    archivo_bar_l = os.path.join(barras_path, f"lcoe_{capacidad}kW_bar.png")
    plt.savefig(archivo_bar_l, dpi=300)
    print(f"✅ Guardado barras de LCOE: {archivo_bar_l}")
    plt.close()


✅ Guardado gráfico de línea (promedio): /home/miguelo/molivares/PRUEBA 1/graficos PV/lineas/lcoe_vs_cap_lat_-10.png
✅ Guardado gráfico de línea (promedio): /home/miguelo/molivares/PRUEBA 1/graficos PV/lineas/lcoe_vs_cap_lat_-5.png
✅ Guardado gráfico de línea (promedio): /home/miguelo/molivares/PRUEBA 1/graficos PV/lineas/lcoe_vs_cap_lat.png
✅ Guardado gráfico de línea (promedio): /home/miguelo/molivares/PRUEBA 1/graficos PV/lineas/lcoe_vs_cap_lat_+5.png
✅ Guardado gráfico de línea (promedio): /home/miguelo/molivares/PRUEBA 1/graficos PV/lineas/lcoe_vs_cap_lat_+10.png


/tmp/ipykernel_22391/953314469.py:46: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["Tilt_type", "País"])[["Energía_MWh", "LCOE_$_kWh"]]


✅ Guardado barras de Energía: /home/miguelo/molivares/PRUEBA 1/graficos PV/barras/energia_100kW_bar.png
✅ Guardado barras de LCOE: /home/miguelo/molivares/PRUEBA 1/graficos PV/barras/lcoe_100kW_bar.png


/tmp/ipykernel_22391/953314469.py:46: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["Tilt_type", "País"])[["Energía_MWh", "LCOE_$_kWh"]]


✅ Guardado barras de Energía: /home/miguelo/molivares/PRUEBA 1/graficos PV/barras/energia_500kW_bar.png
✅ Guardado barras de LCOE: /home/miguelo/molivares/PRUEBA 1/graficos PV/barras/lcoe_500kW_bar.png


...

✅ Guardado barras de Energía: /home/miguelo/molivares/PRUEBA 1/graficos PV/barras/energia_1000kW_bar.png
✅ Guardado barras de LCOE: /home/miguelo/molivares/PRUEBA 1/graficos PV/barras/lcoe_1000kW_bar.png


/tmp/ipykernel_22391/953314469.py:46: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["Tilt_type", "País"])[["Energía_MWh", "LCOE_$_kWh"]]


✅ Guardado barras de Energía: /home/miguelo/molivares/PRUEBA 1/graficos PV/barras/energia_5000kW_bar.png
✅ Guardado barras de LCOE: /home/miguelo/molivares/PRUEBA 1/graficos PV/barras/lcoe_5000kW_bar.png


/tmp/ipykernel_22391/953314469.py:46: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["Tilt_type", "País"])[["Energía_MWh", "LCOE_$_kWh"]]


✅ Guardado barras de Energía: /home/miguelo/molivares/PRUEBA 1/graficos PV/barras/energia_10000kW_bar.png
✅ Guardado barras de LCOE: /home/miguelo/molivares/PRUEBA 1/graficos PV/barras/lcoe_10000kW_bar.png


In [26]:
import PySAM.TcsmoltenSalt as TCSMS
import PySAM.Lcoefcr     as Lcoefcr
import pandas as pd
import os

# Parámetros financieros para LCOE con Lcoefcr
fixed_charge_rate       = 0.07    # 7%
fixed_operating_cost    = 50_000  # $/año
variable_operating_cost = 0.01    # $/kWh

# Función para simular CSP con Molten Salt
def simular_csp(pais, archivo, potencia, tshours):
    try:
        modelo = TCSMS.default("MSPTSingleOwner")
        modelo.SolarResource.assign({"solar_resource_file": archivo})
        modelo.SystemDesign.P_ref   = potencia / 1000  # kW → MW
        modelo.SystemDesign.tshours = tshours

        modelo.execute()
        energia_anual = modelo.Outputs.annual_energy       # kWh
        costo_total  = modelo.Outputs.total_installed_cost # $

        return energia_anual, costo_total

    except Exception as e:
        print(f"⚠️ Error {pais} - {potencia} kW - {tshours} h: {e}")
        return None, None

resultados = []

# Bucle de simulación + cálculo de LCOE con Lcoefcr
for pais, archivo in archivos_recurso.items():
    for tsh in horas_almacenamiento:
        print(f"🔵 Simulando {pais} — {potencia_kw} kW — {tsh} h …")
        energia, costo = simular_csp(pais, archivo, potencia_kw, tsh)

        if energia is None or costo is None:
            lcoe = None
        else:
            modelo_lcoe = Lcoefcr.new()
            modelo_lcoe.SimpleLCOE.annual_energy           = energia
            modelo_lcoe.SimpleLCOE.capital_cost            = costo
            modelo_lcoe.SimpleLCOE.fixed_charge_rate       = fixed_charge_rate
            modelo_lcoe.SimpleLCOE.fixed_operating_cost    = fixed_operating_cost
            modelo_lcoe.SimpleLCOE.variable_operating_cost = variable_operating_cost
            modelo_lcoe.execute()
            lcoe = modelo_lcoe.Outputs.lcoe_fcr

        resultados.append({
            "País": pais,
            "Capacidad_kW": potencia_kw,
            "Horas_almacenamiento": tsh,
            "Energia_anual_kWh": energia,
            "LCOE_$_kWh": lcoe
        })

# Guardar resultados
df_csp     = pd.DataFrame(resultados)
output_csv = os.path.join(carpeta_resultados_csp, "resultados_csp.csv")
df_csp.to_csv(output_csv, index=False)

print(f"\n📁 Resultados CSP guardados en: {output_csv}")


🔵 Simulando ESP — 100000 kW — 4 h …
🔵 Simulando ESP — 100000 kW — 5 h …
🔵 Simulando ESP — 100000 kW — 6 h …
🔵 Simulando ESP — 100000 kW — 7 h …
🔵 Simulando ESP — 100000 kW — 8 h …
🔵 Simulando ESP — 100000 kW — 9 h …
🔵 Simulando ESP — 100000 kW — 10 h …
🔵 Simulando ESP — 100000 kW — 11 h …
🔵 Simulando ESP — 100000 kW — 12 h …
🔵 Simulando ESP — 100000 kW — 13 h …
🔵 Simulando ESP — 100000 kW — 14 h …
🔵 Simulando ESP — 100000 kW — 15 h …
🔵 Simulando ESP — 100000 kW — 16 h …
🔵 Simulando ESP — 100000 kW — 17 h …
🔵 Simulando ESP — 100000 kW — 18 h …
🔵 Simulando CHL — 100000 kW — 4 h …
🔵 Simulando CHL — 100000 kW — 5 h …
🔵 Simulando CHL — 100000 kW — 6 h …
🔵 Simulando CHL — 100000 kW — 7 h …
🔵 Simulando CHL — 100000 kW — 8 h …
🔵 Simulando CHL — 100000 kW — 9 h …
🔵 Simulando CHL — 100000 kW — 10 h …
🔵 Simulando CHL — 100000 kW — 11 h …
🔵 Simulando CHL — 100000 kW — 12 h …
🔵 Simulando CHL — 100000 kW — 13 h …
🔵 Simulando CHL — 100000 kW — 14 h …
🔵 Simulando CHL — 100000 kW — 15 h …
🔵 Simulando C

S_decode ==> Please fix the hour: -2
S_decode ==> Please fix the hour: -1
S_decode ==> Please fix the hour: -2
S_decode ==> Please fix the hour: -1
S_decode ==> Please fix the hour: -2
S_decode ==> Please fix the hour: -1
S_decode ==> Please fix the hour: -2
S_decode ==> Please fix the hour: -1


🔵 Simulando CHN — 100000 kW — 9 h …
🔵 Simulando CHN — 100000 kW — 10 h …
🔵 Simulando CHN — 100000 kW — 11 h …
🔵 Simulando CHN — 100000 kW — 12 h …
🔵 Simulando CHN — 100000 kW — 13 h …
🔵 Simulando CHN — 100000 kW — 14 h …


S_decode ==> Please fix the hour: -2
S_decode ==> Please fix the hour: -1
S_decode ==> Please fix the hour: -2
S_decode ==> Please fix the hour: -1
S_decode ==> Please fix the hour: -2
S_decode ==> Please fix the hour: -1
S_decode ==> Please fix the hour: -2
S_decode ==> Please fix the hour: -1
S_decode ==> Please fix the hour: -2
S_decode ==> Please fix the hour: -1
S_decode ==> Please fix the hour: -2
S_decode ==> Please fix the hour: -1


🔵 Simulando CHN — 100000 kW — 15 h …
🔵 Simulando CHN — 100000 kW — 16 h …
🔵 Simulando CHN — 100000 kW — 17 h …
🔵 Simulando CHN — 100000 kW — 18 h …


S_decode ==> Please fix the hour: -2
S_decode ==> Please fix the hour: -1
S_decode ==> Please fix the hour: -2
S_decode ==> Please fix the hour: -1
S_decode ==> Please fix the hour: -2
S_decode ==> Please fix the hour: -1
S_decode ==> Please fix the hour: -2
S_decode ==> Please fix the hour: -1



📁 Resultados CSP guardados en: /home/miguelo/molivares/PRUEBA 1/resultados_csp.csv


S_decode ==> Please fix the hour: -2
S_decode ==> Please fix the hour: -1


In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Leer resultados CSP (ya incluyen columna "LCOE_$_kWh")
df = pd.read_csv(archivo_resultados_csp)

# Estilos (marcadores) por país
estilos = {
    "ESP": {"marker": "o"},
    "CHL": {"marker": "s"},
    "CHN": {"marker": "D"}
}

# Gráfico único LCOE vs Horas de almacenamiento
plt.figure(figsize=(12,7))
for pais in df["País"].unique():
    df_p = df[df["País"] == pais]
    # Agrupar para una sola línea limpia
    df_line = df_p.groupby("Horas_almacenamiento")["LCOE_$_kWh"] \
                  .mean().reset_index()
    plt.plot(
        df_line["Horas_almacenamiento"],
        df_line["LCOE_$_kWh"],
        label=pais,
        linestyle="--",
        marker=estilos[pais]["marker"],
        linewidth=2
    )

plt.title("LCOE CSP vs Horas de Almacenamiento")
plt.xlabel("Horas de Almacenamiento [h]")
plt.ylabel("LCOE [$/kWh]")
plt.grid(True, ls=":")
plt.legend(title="País")
plt.tight_layout()

# Guardar
ruta_grafico = os.path.join(carpeta_graficos_csp,
                            "lcoe_csp_vs_horas_tres_paises.png")
plt.savefig(ruta_grafico, dpi=300)
plt.close()

print(f"✅ Gráfico guardado en: {ruta_grafico}")


✅ Gráfico guardado en: /home/miguelo/molivares/PRUEBA 1/graficos_csp_lcoe/lcoe_csp_vs_horas_tres_paises.png


In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# === Leer datos ===
df_pv  = pd.read_csv(ruta_pv)
df_csp = pd.read_csv(ruta_csp)

# En tu CSV PV ‘resultados_configuracion.csv’ la energía viene en MWh,
# y la columna se llama “Energía_MWh” y la capacidad “Capacidad_kW”,
# así que creamos las columnas que nuestro script espera:
df_pv["Annual_Energy_kWh"] = df_pv["Energía_MWh"] * 1000
df_pv["Capacity_kW"]       = df_pv["Capacidad_kW"]

# === Costos por país ===
costos_pv = {
    "ESP": {"Capex": 1000, "Opex": 15},
    "CHL": {"Capex": 1000, "Opex": 15},
    "CHN": {"Capex": 1000, "Opex": 15},
}
costos_csp = {
    "ESP": {"Capex": 7000, "Opex": 70},
    "CHL": {"Capex": 7000, "Opex": 70},
    "CHN": {"Capex": 7000, "Opex": 70},
}

# === Función de LCOE ===
def calcular_lcoe(capex_kw, opex_kw_yr, fcr, energia_kwh, cap_kw):
    capex_tot = capex_kw * cap_kw
    opex_tot  = opex_kw_yr * cap_kw
    return (capex_tot * fcr + opex_tot) / energia_kwh

# === Barrido de FCR ===
fcr_values = np.linspace(0.065, 0.095, 5)

# === Estilos de línea por país ===
estilos = {
    "ESP": {"color": "tab:red",   "pv":"o", "csp":"s"},
    "CHL": {"color": "tab:blue",  "pv":"o", "csp":"s"},
    "CHN": {"color": "tab:green", "pv":"o", "csp":"s"},
}

plt.figure(figsize=(12,7))

for pais in ["ESP","CHL","CHN"]:
    # --- PV ---
    row_pv = df_pv[df_pv["País"] == pais].iloc[0]
    e_pv   = row_pv["Annual_Energy_kWh"]
    c_pv   = row_pv["Capacity_kW"]
    capex  = costos_pv[pais]["Capex"]
    opex   = costos_pv[pais]["Opex"]
    lcoe_pv = [calcular_lcoe(capex, opex, f, e_pv, c_pv)
               for f in fcr_values]

    # --- CSP (10 h almacenamiento) ---
    row_csp = df_csp[
        (df_csp["País"] == pais) &
        (df_csp["Horas_almacenamiento"] == 10)
    ].iloc[0]
    e_csp   = row_csp["Energia_anual_kWh"]
    c_csp   = row_csp["Capacidad_kW"]
    capex   = costos_csp[pais]["Capex"]
    opex    = costos_csp[pais]["Opex"]
    lcoe_csp = [calcular_lcoe(capex, opex, f, e_csp, c_csp)
                for f in fcr_values]

    # Graficar PV
    plt.plot(
        fcr_values, lcoe_pv,
        label=f"{pais} – PV",
        color=estilos[pais]["color"],
        marker=estilos[pais]["pv"],
        linestyle="--"
    )
    # Graficar CSP
    plt.plot(
        fcr_values, lcoe_csp,
        label=f"{pais} – CSP",
        color=estilos[pais]["color"],
        marker=estilos[pais]["csp"],
        linestyle="-"
    )

plt.title("Sensibilidad del LCOE vs Fixed Charge Rate")
plt.xlabel("Fixed Charge Rate [-]")
plt.ylabel("LCOE [$/kWh]")
plt.grid(True)
plt.legend()
plt.tight_layout()

# Guardar
out_path = os.path.join(ruta_graficos, "sensibilidad_fcr_todos_paises.png")
plt.savefig(out_path, dpi=300)
plt.close()

print(f"✅ Gráfico combinado guardado en: {out_path}")


✅ Gráfico combinado guardado en: /home/miguelo/molivares/PRUEBA 1/graficos_sensibilidad/sensibilidad_fcr_todos_paises.png
